In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from openbb import obb
import warnings
warnings.filterwarnings("ignore")
obb.user.preferences.output_type = "dataframe"

# Quarter-end Rebalancing Analysis

Hypothesis: Institutions need to rebalance stock and bond allocation percentages as the end of a quarter approaches. Therefore, at the end of a quarter in which bonds have risen while stocks have declined, there should be some demand for stocks as institutions sell bonds and buy stocks.

## Set parameters for analysis

In [2]:
# Number of trading days before quarter end to begin asset rebalancing
asset_allocation_period = 5
# Start date, preferably the Last day of a quarter, no earlier than 2002-09-30 (TLT started trading on 2002-07-30)
start_date_string = "2002-09-30"
# End date, preferably the Last day of a quarter that already ended
end_date_string = "2024-12-31"

## Use openbb to get data for TLT and SPY

In [3]:
tlt = obb.equity.price.historical(
    "TLT",
    start_date = start_date_string,
    end_date = end_date_string,
    provider="yfinance"
).close
tlt.name = "tlt"

In [4]:
spy = obb.equity.price.historical(
    "SPY",
    start_date = start_date_string,
    end_date = end_date_string,
    provider="yfinance"
).close
spy.name = "spy"

In [5]:
data = pd.concat([tlt, spy], axis = 1).dropna()

In [6]:
data.index = pd.to_datetime(data.index).tz_localize("US/Eastern")

In [7]:
data

,tlt,spy
date,,
2002-09-30 00:00:00-04:00,90.400002,81.790001
2002-10-01 00:00:00-04:00,88.940002,85.720001
2002-10-02 00:00:00-04:00,89.449997,83.150002
2002-10-03 00:00:00-04:00,89.230003,82.309998
2002-10-04 00:00:00-04:00,89.510002,80.800003
...,...,...
2024-12-24 00:00:00-05:00,87.870003,601.299988
2024-12-26 00:00:00-05:00,87.820000,601.340027
2024-12-27 00:00:00-05:00,87.099998,595.010010


## For each day, determine days before quarter end and quarter to date net changes

In [8]:
# Establish quarter boundaries
data["month"] = data.index.month
data['last_month_of_qtr'] = data.index.month.isin([3,6,9,12])
data['last_day_of_qtr'] =  (data['month'] != data['month'].shift(-1)) & data['last_month_of_qtr']

In [9]:
# Determine how many trading days before quarter end
desc = data.sort_index(ascending=False)
desc['days_before_qtr_end'] = desc.groupby(desc['last_day_of_qtr'].eq(True).cumsum()).cumcount()
data['days_before_qtr_end'] = desc['days_before_qtr_end']

In [10]:
data

,tlt,spy,month,last_month_of_qtr,last_day_of_qtr,days_before_qtr_end
date,,,,,,
2002-09-30 00:00:00-04:00,90.400002,81.790001,9,True,True,0
2002-10-01 00:00:00-04:00,88.940002,85.720001,10,False,False,63
2002-10-02 00:00:00-04:00,89.449997,83.150002,10,False,False,62
2002-10-03 00:00:00-04:00,89.230003,82.309998,10,False,False,61
2002-10-04 00:00:00-04:00,89.510002,80.800003,10,False,False,60
...,...,...,...,...,...,...
2024-12-24 00:00:00-05:00,87.870003,601.299988,12,True,False,4
2024-12-26 00:00:00-05:00,87.820000,601.340027,12,True,False,3
2024-12-27 00:00:00-05:00,87.099998,595.010010,12,True,False,2


### Calculate the daily and quarter to date net changes of TLT and SPY

In [11]:
# Daily log changes for TLT and SPY
data['tlt_change'] = np.log(data['tlt']) - np.log(data['tlt'].shift(1))
data['spy_change'] = np.log(data['spy']) - np.log(data['spy'].shift(1))

In [12]:
# Determine the quarter to date changes in TLT and SPY
data['qtd_tlt_chng'] = data.groupby([data.index.to_period('q')]).tlt_change.cumsum()
data['qtd_spy_chng'] = data.groupby([data.index.to_period('q')]).spy_change.cumsum()

In [13]:
data

,tlt,spy,month,last_month_of_qtr,last_day_of_qtr,days_before_qtr_end,tlt_change,spy_change,qtd_tlt_chng,qtd_spy_chng
date,,,,,,,,,,
2002-09-30 00:00:00-04:00,90.400002,81.790001,9,True,True,0,NaN,NaN,NaN,NaN
2002-10-01 00:00:00-04:00,88.940002,85.720001,10,False,False,63,-0.016282,0.046931,-0.016282,0.046931
2002-10-02 00:00:00-04:00,89.449997,83.150002,10,False,False,62,0.005718,-0.030440,-0.010565,0.016491
2002-10-03 00:00:00-04:00,89.230003,82.309998,10,False,False,61,-0.002462,-0.010154,-0.013027,0.006338
2002-10-04 00:00:00-04:00,89.510002,80.800003,10,False,False,60,0.003133,-0.018516,-0.009894,-0.012178
...,...,...,...,...,...,...,...,...,...,...
2024-12-24 00:00:00-05:00,87.870003,601.299988,12,True,False,4,0.004220,0.011054,-0.110129,0.046883
2024-12-26 00:00:00-05:00,87.820000,601.340027,12,True,False,3,-0.000569,0.000067,-0.110698,0.046949
2024-12-27 00:00:00-05:00,87.099998,595.010010,12,True,False,2,-0.008232,-0.010582,-0.118930,0.036367


In [14]:
data[50:70]

,tlt,spy,month,last_month_of_qtr,last_day_of_qtr,days_before_qtr_end,tlt_change,spy_change,qtd_tlt_chng,qtd_spy_chng
date,,,,,,,,,,
2002-12-10 00:00:00-05:00,86.879997,90.699997,12,True,False,14,0.002651,0.013319,-0.039716,0.103402
2002-12-11 00:00:00-05:00,87.459999,90.779999,12,True,False,13,0.006654,0.000882,-0.033063,0.104284
2002-12-12 00:00:00-05:00,87.379997,90.769997,12,True,False,12,-0.000915,-0.000110,-0.033978,0.104174
2002-12-13 00:00:00-05:00,86.529999,89.339996,12,True,False,11,-0.009775,-0.015880,-0.043753,0.088294
2002-12-16 00:00:00-05:00,85.879997,91.650002,12,True,False,10,-0.007540,0.025528,-0.051293,0.113822
2002-12-17 00:00:00-05:00,85.879997,90.849998,12,True,False,9,0.000000,-0.008767,-0.051293,0.105055
2002-12-18 00:00:00-05:00,86.550003,89.800003,12,True,False,8,0.007771,-0.011625,-0.043522,0.093430
2002-12-19 00:00:00-05:00,87.320000,89.160004,12,True,False,7,0.008857,-0.007152,-0.034665,0.086278
2002-12-20 00:00:00-05:00,87.379997,89.989998,12,True,False,6,0.000687,0.009266,-0.033978,0.095544


# Identify trade setups

We want to identify situations in which TLT is up quarter to date, and SPY is down quater to date, with a specified number of trading days (asset_allocation_period) remaining in the quarter. The theory is that large institutions would need to sell bonds and buy stocks in this scenario.

In [15]:
data['buy_setup_int'] = 0
data.loc[((data['days_before_qtr_end'] == asset_allocation_period)
          & (data['qtd_tlt_chng'] > 0)
          & (data['qtd_spy_chng'] < 0)),
          'buy_setup_int'] = 1

Trade: Buy SPY asset_allocation_period days before quarter end and sell it at the end of the quarter.

In [16]:
# Set target position to 1 (100%) for the holding period after a buy setup
data.loc[data['buy_setup_int'].rolling(asset_allocation_period).sum() >= 1, 'target_position'] = 1.00
data.loc[data['buy_setup_int'].rolling(asset_allocation_period).sum() < 1, 'target_position'] = 0.00

In [17]:
data.loc[data['target_position'] == 1.00]

,tlt,spy,month,last_month_of_qtr,last_day_of_qtr,days_before_qtr_end,tlt_change,spy_change,qtd_tlt_chng,qtd_spy_chng,buy_setup_int,target_position
date,,,,,,,,,,,,
2004-03-24 00:00:00-05:00,91.089996,109.550003,3,True,False,5,-0.000549,0.000822,0.063917,-0.015668,1,1.0
2004-03-25 00:00:00-05:00,90.739998,111.000000,3,True,False,4,-0.003850,0.013149,0.060067,-0.002519,0,1.0
2004-03-26 00:00:00-05:00,89.779999,111.029999,3,True,False,3,-0.010636,0.000270,0.049431,-0.002249,0,1.0
2004-03-29 00:00:00-05:00,89.180000,112.589996,3,True,False,2,-0.006705,0.013952,0.042725,0.011703,0,1.0
2004-03-30 00:00:00-05:00,89.120003,112.970001,3,True,False,1,-0.000673,0.003369,0.042052,0.015073,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-24 00:00:00-04:00,162.910004,243.149994,3,True,False,5,-0.018790,0.086731,0.184374,-0.280438,1,1.0
2020-03-25 00:00:00-04:00,162.539993,246.789993,3,True,False,4,-0.002274,0.014859,0.182100,-0.265579,0,1.0
2020-03-26 00:00:00-04:00,163.339996,261.200012,3,True,False,3,0.004910,0.056749,0.187010,-0.208830,0,1.0


## Calculate net changes of entire time period (population) and of trades (sample)

In [18]:
# Select the days with buy or sell setups
transactions = data.loc[
    ((data['target_position'] == 1) & (data['target_position'].shift(1) == 0))
    | ((data['target_position'] == 0) & (data['target_position'].shift(1) == 1))][['spy', 'days_before_qtr_end']]

In [19]:
# Calculate forward return
transactions['exit_spy'] = transactions['spy'].shift(-1)
transactions['y_log_return'] = np.log(transactions['spy'].shift(-1)) - np.log(transactions['spy'])

In [20]:
transactions

,spy,days_before_qtr_end,exit_spy,y_log_return
date,,,,
2004-03-24 00:00:00-05:00,109.550003,5,113.099998,0.031891
2004-03-31 00:00:00-05:00,113.099998,0,110.949997,-0.019193
2004-09-23 00:00:00-04:00,110.949997,5,111.760002,0.007274
2004-09-30 00:00:00-04:00,111.760002,0,148.130005,0.281737
2007-12-21 00:00:00-05:00,148.130005,5,146.210007,-0.013046
2007-12-31 00:00:00-05:00,146.210007,0,134.720001,-0.081845
2008-03-24 00:00:00-04:00,134.720001,5,131.970001,-0.020624
2008-03-31 00:00:00-04:00,131.970001,0,118.550003,-0.107240
2008-09-23 00:00:00-04:00,118.550003,5,115.989998,-0.021831


In [21]:
trades = transactions.loc[transactions['days_before_qtr_end'] == asset_allocation_period].dropna()

In [22]:
trades

,spy,days_before_qtr_end,exit_spy,y_log_return
date,,,,
2004-03-24 00:00:00-05:00,109.550003,5,113.099998,0.031891
2004-09-23 00:00:00-04:00,110.949997,5,111.760002,0.007274
2007-12-21 00:00:00-05:00,148.130005,5,146.210007,-0.013046
2008-03-24 00:00:00-04:00,134.720001,5,131.970001,-0.020624
2008-09-23 00:00:00-04:00,118.550003,5,115.989998,-0.021831
2008-12-23 00:00:00-05:00,86.160004,5,90.239998,0.046267
2010-06-23 00:00:00-04:00,109.230003,5,103.220001,-0.056593
2011-06-23 00:00:00-04:00,128.300003,5,131.970001,0.028203
2011-09-23 00:00:00-04:00,113.540001,5,113.150002,-0.003441


### Population statistics

In [23]:
# Get log returns for the entire population based on the holding period
population = data.iloc[::asset_allocation_period][['spy', 'buy_setup_int']]

In [24]:
# Calculate forward return in the population
population['exit_spy'] = population['spy'].shift(-1)
population['y_log_return'] = np.log(population['spy'].shift(-1)) - np.log(population['spy'])

In [25]:
population

,spy,buy_setup_int,exit_spy,y_log_return
date,,,,
2002-09-30 00:00:00-04:00,81.790001,0,79.129997,-0.033063
2002-10-07 00:00:00-04:00,79.129997,0,84.629997,0.067197
2002-10-14 00:00:00-04:00,84.629997,0,90.169998,0.063408
2002-10-21 00:00:00-04:00,90.169998,0,89.610001,-0.006230
2002-10-28 00:00:00-05:00,89.610001,0,91.129997,0.016820
...,...,...,...,...
2024-11-29 00:00:00-05:00,602.549988,0,607.809998,0.008692
2024-12-06 00:00:00-05:00,607.809998,0,604.210022,-0.005940
2024-12-13 00:00:00-05:00,604.210022,0,591.150024,-0.021852


## Calculate backtest statistics

In [26]:
trades['y_log_return'].describe()

count    14.000000
mean      0.008190
std       0.031358
min      -0.056593
25%      -0.012342
50%       0.009289
75%       0.030969
max       0.058312
Name: y_log_return, dtype: float64

In [27]:
print("The sample mean log return is ", trades['y_log_return'].mean())
print("The population mean log return is ", population['y_log_return'].mean())

The sample mean log return is  0.00819037447929454
The population mean log return is  0.001761558910567375


In [28]:
print("Backtest sample statistics (log returns):")
print("Mean: ", trades['y_log_return'].mean())
print("Standard deviation: ", trades['y_log_return'].std())
print("N: ", trades['y_log_return'].count())
print("t: ", (trades['y_log_return'].mean() - population['y_log_return'].mean()) /
      (trades['y_log_return'].std() / (trades['y_log_return'].count() ** 0.5)))

Backtest sample statistics (log returns):
Mean:  0.00819037447929454
Standard deviation:  0.03135760963681416
N:  14
t:  0.7671000927537273


In [29]:
trades

,spy,days_before_qtr_end,exit_spy,y_log_return
date,,,,
2004-03-24 00:00:00-05:00,109.550003,5,113.099998,0.031891
2004-09-23 00:00:00-04:00,110.949997,5,111.760002,0.007274
2007-12-21 00:00:00-05:00,148.130005,5,146.210007,-0.013046
2008-03-24 00:00:00-04:00,134.720001,5,131.970001,-0.020624
2008-09-23 00:00:00-04:00,118.550003,5,115.989998,-0.021831
2008-12-23 00:00:00-05:00,86.160004,5,90.239998,0.046267
2010-06-23 00:00:00-04:00,109.230003,5,103.220001,-0.056593
2011-06-23 00:00:00-04:00,128.300003,5,131.970001,0.028203
2011-09-23 00:00:00-04:00,113.540001,5,113.150002,-0.003441
